In [ ]:
import os
import requests
from PIL import Image

from tqdm import tqdm

In [ ]:
def is_row_white(image, y):
    for x in range(image.width):
        if image.getpixel((x, y)) != (255, 255, 255):  # Not white
            return False
    return True

def crop_top(image):
    for y in range(image.height):
        if not is_row_white(image, y):
            break
    if y == image.height-1:
        raise Exception("y reached end")
    for yy in range(y+1, image.height):
        if is_row_white(image,yy):
            break
    if yy == image.height-1:
        raise Exception(f"yy reached end, y was {y}, height was {image.height}")

    crop_area = (0, yy, image.width, image.height)
    cropped_image = image.crop(crop_area)
    return cropped_image

display(crop_top(Image.open("images_split/20240101_173712_YES.jpg")))

In [ ]:
left_fails = []
right_fails = []

for filename in tqdm(os.listdir("images")):
    img = Image.open(os.path.join("images",filename))
    middle = img.width // 2

    left_half = img.crop((0, 0, middle, img.height))
    right_half = img.crop((middle, 0, img.width, img.height))

    name = filename[:-4]

    try:
        left_cropped = crop_top(left_half)
        left_cropped.save(f"images_split/{name}_YES.jpg")
    except:
        print(f"Couldn't do left for file: {name}")
        left_fails.append(name)

    try:
        right_cropped = crop_top(right_half)
        right_cropped.save(f"images_split/{name}_BUT.jpg")
    except:
        print(f"Couldn't do right for file: {name}")
        right_fails.append(name)

In [ ]:
!mkdir split_fails
for name in left_fails:
    ! cp images/{name}.jpg split_fails/{name}_YES.jpg
for name in right_fails:
    ! cp images/{name}.jpg split_fails/{name}_BUT.jpg

In [ ]:
!zip -r split_fails.zip split_fails